In [ ]:
import warnings
import logging
from data_loaders import load_and_merge
from forecast_helpers import ForecastConfig
from forecast_eval import backtest_forecast, naive_baseline_forecast
import matplotlib.pyplot as plt
import pandas as pd

# Suppress warnings and pandas settings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option("display.precision", 2)

# Configure logging - remove duplicate handlers and prevent propagation
logging.basicConfig(level=logging.INFO, format="%(message)s", force=True)
logger = logging.getLogger(__name__)
logger.handlers.clear()  # Clear any existing handlers
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
logger.addHandler(console_handler)
logger.propagate = False  # Prevent propagation to root logger

# Load data and define evaluation period
df = load_and_merge()
test_years = [2022, 2023, 2024]

# Model configuration
base_config = ForecastConfig(
    MAX_KNOTS_TOTAL=6,
    MAX_KNOTS_SHARES=4,
    ALPHA_TOTAL=0.005,
    TAU_YEARS=5.0,
    W_HIST_MAX=0.4,
    W_UNIF_MAX=0.2,
    MOM_TILT_MAX=0.7,
    MOM_TAU=3,
    RECENT_WIN=7,
    CAT_BOOST_SURGE=7,
    GROWTH_FLOOR_FRAC=0.07,
    SHARE_KAPPA=35,
    # from calibration_analysis.ipynb
    B_SIM=5000,                    # Increased for stability
    CONFIDENCE_LEVEL=0.90,         
    BOOTSTRAP_ITERS=500,           
    MIN_PI_WIDTH=0.2,              
    GROWTH_UNCERTAINTY_FACTOR=1,  # Calibrated value 0.686,
    YTD_YEAR=2025,
    MONOTONE_TOTALS=False,
    NO_DIP_FIRST_YEAR=False,
    NONDECREASING_CATEGORIES=False,
    ASSIMILATE_YTD=False # True for production
)

# Log configuration - modify the logging section
logger.info("\nRunning backtest with config:")
for k, v in sorted(base_config.__dict__.items()):
    if not k.startswith("_"):  # Only log public attributes
        logger.info(f"- {k}: {v}")

# Run models
model_results = backtest_forecast(df, "date", "Risk Domain", test_years, base_config)
baseline_results = naive_baseline_forecast(df, "date", test_years)


def summarize_results(results, model_name):
    """Create DataFrame summarizing model results.

    Args:
        results: List of BacktestResult objects
        model_name: Name of the model for identification

    Returns:
        DataFrame with performance metrics
    """
    return pd.DataFrame(
        {
            "Year": [r.year for r in results],
            "Actual": [r.actual for r in results],
            "Predicted": [r.predicted for r in results],
            "MAE": [r.mae for r in results],
            "MAPE": [r.mape for r in results],
            "RMSE": [r.rmse for r in results],
            "Model": model_name,
        }
    )


# Combine results
results_df = pd.concat(
    [
        summarize_results(model_results, "Main Model"),
        summarize_results(baseline_results, "Baseline"),
    ]
)

# Visualization
plt.figure(figsize=(10, 6))
for model in ["Main Model", "Baseline"]:
    mask = results_df["Model"] == model
    plt.plot(
        results_df[mask]["Year"],
        results_df[mask]["Predicted"],
        "o-",
        label=f"{model} prediction",
    )
plt.plot(
    results_df["Year"].unique(),
    results_df[results_df["Model"] == "Main Model"]["Actual"],
    "k--",
    label="Actual",
)
plt.legend()
plt.title("Forecast vs Actual: Model Comparison")
plt.grid(True, alpha=0.3)
plt.show()

# Performance summary
print("\n" + "=" * 50)
print("MODEL PERFORMANCE SUMMARY")
print("=" * 50)

print("\nAverage Error Metrics:")
print(results_df.groupby("Model")[["MAE", "MAPE", "RMSE"]].mean())

print("\nDetailed Results by Year:")
print(results_df.sort_values(["Year", "Model"]))

# Main model error analysis
model_metrics = results_df[results_df["Model"] == "Main Model"][
    ["Year", "Actual", "Predicted"]
]
model_metrics["Error %"] = (
    (model_metrics["Predicted"] - model_metrics["Actual"])
    / model_metrics["Actual"]
    * 100
)
print("\nMain Model Yearly Errors:")
print(model_metrics.to_string(index=False))


def calculate_adaptive_metrics(results_df):
    """Calculate growth-adjusted performance metrics.

    Args:
        results_df: DataFrame containing model results

    Returns:
        float: Weighted error accounting for growth rates
    """
    main_model = results_df[results_df["Model"] == "Main Model"].copy()
    main_model["Growth_Rate"] = main_model["Actual"].pct_change()
    main_model["Weighted_Error"] = main_model["MAPE"] * (1 + main_model["Growth_Rate"])
    return main_model["Weighted_Error"].mean()